# Projet TMDB Movies – Notes complètes

## 1️⃣ Objectif du projet
- Construire un pipeline **Bronze → Silver → Gold** sur Databricks avec Delta Lake.
- Objectif final : tables **BI-ready** pour dashboards et analyses.
- Pipeline capable de gérer :
  - CSV volumineux (~930k films)
  - Titres avec **virgules, guillemets ou retours à la ligne**
  - Merge incrémental
  - Time Travel Delta pour restaurer d’anciennes versions

---

## 2️⃣ Structure du pipeline

| Couche   | Contenu                                                | Objectif                                         |
|----------|--------------------------------------------------------|-------------------------------------------------|
| **Bronze** | Données brutes de tous les CSV, toutes les colonnes   | Historisation complète, jamais modifiées       |
| **Silver** | Données nettoyées, typées, filtrées, merge incrémental | Source de vérité pour analyses et Gold         |
| **Gold**   | Tables BI-ready, agrégées ou détaillées              | Dashboards et rapports BI optimisés            |

---

## 3️⃣ Étapes réalisées

### 3.1 Ingestion Bronze
- Lecture automatique de tous les CSV dans le volume `/Volumes/workspace/demo/movies/`.
- Options Spark pour gérer correctement les titres contenant des **virgules ou retours à la ligne** :
  - `header=True`
  - `multiLine=True`
  - `escape='"'`
  - `quote='"'`
  - `ignoreLeadingWhiteSpace=True`
- Fusion de tous les CSV avec `unionByName`.
- Écriture en Delta table Bronze :
  - `df_bronze.write.format("delta").mode("overwrite").saveAsTable(BRONZE_TABLE)`
- **Time Travel** possible pour revenir à n’importe quelle version précédente.

---

### 3.2 Transformation Silver
- Fonction `clean_tmdb(df)` :
  - Filtre `title IS NOT NULL`
  - Typage :
    - `vote_average` → double
    - `release_date` → date
- Gestion des **ID uniques** :
  - Si `id` absent, création automatique : `mon_id = concat(title, release_date)`
- Merge incrémental Delta pour ajouter uniquement les nouvelles lignes.
- Comptage du nombre de lignes insérées avant/après le merge pour vérifier les nouvelles données.

---

### 3.3 Création Gold
- Gold peut contenir :
  - **Toutes les colonnes utiles**, sauf : `backdrop_path`, `imdb_id`, `poster_path`, `keywords`
  - Agrégations possibles : `release_year`, `movie_count`, `avg_rating`
- Gestion des erreurs avec **try/except**
- Optimisation Delta pour BI :
  - `OPTIMIZE + ZORDER` pour accélérer les filtres et agrégations
- Solutions pour éviter les erreurs de schéma :
  - Supprimer la table Gold avant création (`DROP TABLE IF EXISTS`)
  - Ou `overwriteSchema=True` pour forcer la compatibilité

---

### 3.4 Bonnes pratiques et recommandations
- **Bronze = raw**, jamais modifié.
- **Silver = nettoyé, typé, source de vérité**, idéal pour créer des **dimensions et tables fact**
- **Gold = BI-ready**, tables détaillées ou agrégées, optimisées pour dashboards.
- Créer des **dimensions** : `dim_movie`, `dim_genre`, `dim_language`.
- Créer des **tables fact** : `fact_movie_rating` pour votes et notes.
- Utiliser **merge incrémental** pour ajouter seulement les nouvelles données.
- Utiliser **Time Travel** pour revenir à une version précédente si nécessaire.
- Optimiser Gold avec `OPTIMIZE + ZORDER` pour des filtres rapides.

---

### 3.5 Gestion des CSV problématiques
- Titres avec **virgules, guillemets ou retours à la ligne** ne cassent plus le pipeline grâce aux options Spark.
- Colonnes correctement remises dans l’ordre après lecture.
- Silver et Gold ne perdent aucune donnée essentielle.

---

### 3.6 Avantages de ce pipeline
- **Fiable et robuste** : merge incrémental, gestion des IDs uniques, Time Travel
- **Extensible** : possibilité de créer d’autres dimensions ou KPIs
- **BI-ready** : tables Gold optimisées pour dashboards
- **Pro** : conforme aux bonnes pratiques Data Lake / Data Warehouse sur Databricks

---

## 4️⃣ Recommandation globale
- **Bronze** → données brutes
- **Silver** → données nettoyées et typées, dimensions et faits
- **Gold** → tables BI-ready, détaillées ou agrégées
- **Time Travel Delta** → revenir à une version précédente
- **Merge incrémental** → ajouter uniquement les nouvelles lignes
